In [1]:
import tensorflow as tf

In [2]:
# extract data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# simple softmax regression model creation 
x = tf.placeholder(tf.float32, shape=[None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# cross entropy for training 
c_e = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(c_e)

In [4]:
# start the tensorflow session and train the model

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# for reference, the _ here indicates that the loop variable is not needed and tells python not to care about holding it
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch_xs, y_: batch_ys})

In [5]:
# Check the accuracy of our created and trained model

# tf.argmax returns the index of the highest entry in a tensor along some axis 
# (gives us the soft max values from the predicted values and the one-hot values from the correct labels)
#
# this gives us back the indices where correct prediction occurs and incorrect
# for example
# [correct, correct, incorrect] returns as [1, 1, 0]
# this array can be used to calc the overall percentage 
#
# TLDR;
# Converts from one-hot form of data into a simple class labels list for both predicted and correct datasets
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
# Gauge the accuracy of the model against the test data

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.918


# Improving upon the simple one layer NN structure to improve accuracy

Simple one layer NN gets ~92% accuracy with the MNIST handwritten digits. In order to improve this accuracy, creating a convolutional NN which captures visual features in a more sophisticated way and is better suited for image deep learning tasks than simple softmax. 

## Convolutional Neural Network

In [7]:
# Creating some helping functions for weight and bias creation for our NN
def weight_variable(shape):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init)

def bias_variable(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

In [8]:
# Create helper functions for convolution and pooling. 
# For more information about CNN:
# https://en.wikipedia.org/wiki/Convolutional_neural_network

# This helper function lets us encapsulate our stride and padding settings basically and just focus on shape
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# pooling for every 2x2 space into 1 output based on the max value in the 2x2 square
# Example:
#     [[1, 2],
#     [3, 4]]
#     would return [4]
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
# reshaping of input images. Input images 28*28. Flatten them. 
x = tf.placeholder(tf.float32, shape=[None, 784])
x_flat = tf.reshape(x, [-1, 28, 28, 1])

# First layer will be 5*5 patches with 1 input channel (greyscale images). Output of 32 features for each patch
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# create first layer of convolution
# relu is a type of activation function 
# https://en.wikipedia.org/wiki/Activation_function
h_conv1 = tf.nn.relu(conv2d(x_flat, W_conv1) + b_conv1)

# pooling follows first layer 
h_pool1 = max_pool_2x2(h_conv1)

# second layer

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
# Add densely connected layer at the end before softmax 
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [11]:
# Drop-out applied before the readout layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [12]:
# Final softmax output layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [13]:
c_e = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(c_e)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch_xs, batch_ys = mnist.train.next_batch(50)
    if i % 1000 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch_xs, y_:batch_ys, keep_prob: 1.0
        })
        print("Step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x:batch_xs, y_:batch_ys, keep_prob: 0.5})

Step 0, training accuracy 0.16
Step 1000, training accuracy 0.94
Step 2000, training accuracy 1
Step 3000, training accuracy 0.96
Step 4000, training accuracy 1
Step 5000, training accuracy 1
Step 6000, training accuracy 0.98
Step 7000, training accuracy 0.98
Step 8000, training accuracy 1
Step 9000, training accuracy 0.98
Step 10000, training accuracy 1
Step 11000, training accuracy 0.98
Step 12000, training accuracy 1
Step 13000, training accuracy 1
Step 14000, training accuracy 1
Step 15000, training accuracy 1
Step 16000, training accuracy 1
Step 17000, training accuracy 1
Step 18000, training accuracy 1
Step 19000, training accuracy 1


In [14]:
print('test accuracy: %g'%accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0}))

test accuracy: 0.9918
